In [25]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
from IPython.display import HTML
np.random.seed(0)

# Parameters
L = 20  # Length of domain
N = 256  # Number of grid points
dx = L / N  # Grid spacing
x = np.linspace(0, L, N, endpoint=False)  # Spatial grid

# Function to create sinusoidal initial condition
def create_initial_condition():
    k1, k2 = 2, 1  # Wavenumbers
    phase1, phase2 = np.random.rand() * 2 * np.pi, np.random.rand() * 2 * np.pi
    amplitude1, amplitude2 = np.random.rand(), np.random.rand() * 0.5
    return amplitude1 * np.sin(2 * np.pi * k1 * x / L + phase1) + amplitude2 * np.sin(2 * np.pi * k2 * x / L + phase2)

# Define the Kuramoto-Sivashinsky equation
def kuramoto_sivashinsky(t, u):
    uxx = np.roll(u, 1) - 2 * u + np.roll(u, -1)
    uxxxx = np.roll(u, 2) - 4 * np.roll(u, 1) + 6 * u - 4 * np.roll(u, -1) + np.roll(u, -2)
    dudt = - uxx - uxxxx - 0.5 * (np.roll(u, -1) * np.roll(u, 1) - np.roll(u, -1) * np.roll(u, -2))
    return dudt

# Time grid
t_span = np.linspace(0, 50, 1000)

# Solve the ODE system for multiple initial conditions
all_solutions = []
num_initial_conditions = 500

for _ in range(num_initial_conditions):
    u0 = create_initial_condition()
    sol = solve_ivp(kuramoto_sivashinsky, [t_span[0], t_span[-1]], u0, t_eval=t_span, method='RK45')
    all_solutions.append(sol.y.T)

all_solutions = np.array(all_solutions)
print(all_solutions.shape)


(500, 1000, 256)


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
nn_input = scaler.fit_transform(all_solutions[:, 1:])
nn_output = scaler.transform(all_solutions[:, :-1])
X_train, X_test, Y_train, Y_test = train_test_split(nn_input, nn_output, test_size=0.3)

print(X_train.shape, Y_train.shape)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
fig, ax = plt.subplots()
line, = ax.plot(x, nn_input[0])
line2, = ax.plot(x, nn_output[0])
plt.close()

def update(frame):
    line.set_ydata(nn_input[frame])
    line2.set_ydata(nn_output[frame])
    return line, line2, 

ani = FuncAnimation(fig, update, frames=len(t_span)-1, blit=True, interval=100)
HTML(ani.to_jshtml())

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras import optimizers

# Define a function to create the Keras model
def create_model(learning_rate, dropout_rate, neurons1, neurons2):
    model = Sequential()
    model.add(Input(shape=(N,)))
    model.add(Dense(neurons1, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(256, activation='linear'))
    
    # Compile the model
    adam_optimizer = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=adam_optimizer, loss='mean_squared_error')
    return model

In [18]:
from scikeras.wrappers import KerasRegressor

# Wrap the Keras model with KerasRegressor
model = KerasRegressor(model=create_model, verbose=1, learning_rate=0.0001, epochs=200, dropout_rate=0.1, neurons1=256, neurons2=256, batch_size=256)

# Fit the grid search
best_model = model.fit(X_train, Y_train)

Epoch 1/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.2621
Epoch 2/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0708
Epoch 3/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.0609
Epoch 4/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0567
Epoch 5/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0542
Epoch 6/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.0525
Epoch 7/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0513
Epoch 8/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0502
Epoch 9/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0495
Epoch 10/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0488
Epoch 11/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0484
Epoch 12/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0478
Epoch 13/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0475
Epoch 14/200
5469/5469 ━━━━━━━━━━━━━━━━━━━━ 937s 171ms/step - loss: 0.047

In [24]:
score = best_model.score(X_test, Y_test)
print(f'Score: {score}')

2344/2344 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
Score: 0.9865871644381135


In [21]:
u0 = create_initial_condition().reshape(1, -1)
u_guess = best_model.predict(u0)
sol = []
for _ in range(len(t_span)):
    sol.append(u_guess)
    u_guess = best_model.predict(u_guess)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━

In [22]:
act_sol = solve_ivp(kuramoto_sivashinsky, [t_span[0], t_span[-1]], u0.flatten(), t_eval=t_span, method='RK45').y
print("Network output", np.array(sol).shape)
print("Actual solution", act_sol.shape)

Network output (1000, 1, 256)
Actual solution (256, 1000)


In [ ]:
fig, ax = plt.subplots()
line, = ax.plot(x, act_sol[:, 0], label='Actual')
line2, = ax.plot(x, sol[0].flatten(), label='Predicted')
plt.close()

def update(frame):
    line.set_ydata(act_sol[:, frame])
    line2.set_ydata(sol[frame].flatten())
    return line, line2,

ani = FuncAnimation(fig, update, frames=len(t_span), blit=True, interval=100)
HTML(ani.to_jshtml())